## Imports

In [6]:
import numpy as np
import sklearn.metrics as metrics
import matplotlib as plt

## Load training data

In [2]:
X_emb = np.load("processed_train/X_emb.npy", allow_pickle=True)
Xt_emb = np.load("processed_train/Xt_emb.npy", allow_pickle=True)
yr = np.load("processed_train/yr.npy", allow_pickle=True)
ys =np.load("processed_train/ys.npy", allow_pickle=True)
yr = yr.astype('float64') 
ys = ys.astype('float64') 

y = np.concatenate((np.transpose([yr]), np.transpose([ys])), axis=1)

print(f"type(X_emb)-->{type(X_emb)}")
print(f"type(Xt_emb)-->{type(Xt_emb)}")
print(f"type(ys)-->{type(ys)}")
print(f"type(yr)-->{type(yr)}\n")

print(f"y.shape-->{y.shape}")
print(f"type(y)-->{type(y)}\n")

print(f"X_emb.shape-->{X_emb.shape}")
print(f"Xt_emb.shape-->{Xt_emb.shape}")
print(f"ys.shape-->{ys.shape}")
print(f"yr.shape-->{yr.shape}")

type(X_emb)--><class 'numpy.ndarray'>
type(Xt_emb)--><class 'numpy.ndarray'>
type(ys)--><class 'numpy.ndarray'>
type(yr)--><class 'numpy.ndarray'>

y.shape-->(1400, 2)
type(y)--><class 'numpy.ndarray'>

X_emb.shape-->(1400, 864, 300)
Xt_emb.shape-->(200, 864, 300)
ys.shape-->(1400,)
yr.shape-->(1400,)


## LSTM

In [3]:
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, Bidirectional

import keras.backend as K
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_emb, y, test_size=0.1, shuffle= True, random_state=42)


# model initialization
model = Sequential()
model.add(Bidirectional(LSTM(64), input_shape = (X_emb.shape[1], X_emb.shape[2])))
model.add(Dense(24, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
#model.build(input_shape = (64, X_emb.shape[1], X_emb.shape[2]))
model.summary()

2023-10-20 14:09:17.388284: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 14:10:09.414510: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 128)              186880    
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 2)                 50        
                                                                 
Total params: 190,026
Trainable params: 190,026
Non-trainable params: 0
_________________________________________________________________


In [5]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau

es = EarlyStopping(monitor='val_loss', mode='min', verbose=5, patience=4, restore_best_weights=True)

cb_plateau = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1,
    mode="auto",
    min_delta=0.001,
    cooldown=0,
    min_lr=1e-4,
)

bf = model.fit(x=X_train, y=y_train, validation_data = (X_val, y_val), epochs=50, batch_size=32, callbacks=[es, cb_plateau] )

Epoch 1/50
40/40 [==============================] - 62s 2s/step - loss: 0.4535 - accuracy: 0.6865 - val_loss: 0.4800 - val_accuracy: 0.6929
Epoch 2/50
40/40 [==============================] - 57s 1s/step - loss: 0.3717 - accuracy: 0.7103 - val_loss: 0.4002 - val_accuracy: 0.8071
Epoch 3/50
40/40 [==============================] - 44s 1s/step - loss: 0.3199 - accuracy: 0.7365 - val_loss: 0.3481 - val_accuracy: 0.7786
Epoch 4/50
40/40 [==============================] - 58s 1s/step - loss: 0.3019 - accuracy: 0.7246 - val_loss: 0.3309 - val_accuracy: 0.8000
Epoch 5/50
40/40 [==============================] - 70s 2s/step - loss: 0.2432 - accuracy: 0.7492 - val_loss: 0.3510 - val_accuracy: 0.5929
Epoch 6/50
40/40 [==============================] - 68s 2s/step - loss: 0.2034 - accuracy: 0.7706 - val_loss: 0.3008 - val_accuracy: 0.8500
Epoch 7/50
40/40 [==============================] - 73s 2s/step - loss: 0.1597 - accuracy: 0.7825 - val_loss: 0.3062 - val_accuracy: 0.7214
Epoch 8/50
40/40 [==

### Metrics

In [7]:
# best epoch obtained
results_test = model.evaluate(X_val, y_val)
print(results_test)

5/5 [==============================] - 2s 372ms/step - loss: 0.2624 - accuracy: 0.7786
[0.2624445855617523, 0.7785714268684387]


In [10]:
# summary of the model applied to the validation set
y_preds = model.predict(X_val)



for i in range(len(y_preds)):
    for j in range(len(y_preds[i])):
        if y_preds[i,j] > 0.5:
            y_preds[i,j] = 1
        else:
            y_preds[i,j] = 0

print(metrics.classification_report(y_val, y_preds))

print(metrics.multilabel_confusion_matrix(y_val, y_preds))

5/5 [==============================] - 4s 684ms/step
              precision    recall  f1-score   support

           0       0.86      0.82      0.84        73
           1       0.94      0.99      0.96        69

   micro avg       0.90      0.90      0.90       142
   macro avg       0.90      0.90      0.90       142
weighted avg       0.90      0.90      0.90       142
 samples avg       0.70      0.69      0.69       142

[[[57 10]
  [13 60]]

 [[67  4]
  [ 1 68]]]


/Users/nunomachado/opt/anaconda3/envs/LN23/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nunomachado/opt/anaconda3/envs/LN23/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
loss = bf.history['loss']
val_loss = bf.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'r', label = 'Training set')
plt.plot(epochs, val_loss, 'b', label = 'Validation set')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show

NameError: name 'bf' is not defined

In [ ]:
loss = bf.history['accuracy']
val_loss = bf.history['val_accuracy']
epochs = range(1, len(loss) + 1)


plt.plot(epochs, loss, 'r', label = 'Training set')
plt.plot(epochs, val_loss, 'b', label = 'Validation set')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show

## SVM

In [ ]:
from sklearn import svm